In [3]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.26.1 which is incompatible.


In [4]:
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import pywrapcp

In [6]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Données : Matrice des distances entre les 13 wilayas
def create_data_model():
    """Définit la matrice des distances et les paramètres."""
    data = {}
    # Matrice des distances (exemple en kilomètres, indexation 1-13)
    data['distance_matrix'] = [
        [0,   276, 488, 794, 751, 1086, 1518, 1564, 1065, 797, 1823, 1339, 1087],  # 1: Néma
        [276,   0, 212, 519, 476, 810, 1242, 1289, 790, 521, 1547, 1063, 811],      # 2: Aioun
        [488, 212,   0, 310, 263, 599, 1031, 1078, 579, 313, 1336, 852, 600],       # 3: Assaba
        [794, 519, 310,   0, 243, 300, 843, 890, 680, 231, 1148, 664, 412],        # 4: Kaedi
        [751, 476, 263, 243,   0, 335, 767, 813, 436, 474, 1072, 587, 336],        # 5: Brakna
        [1086, 810, 599, 300, 335,   0, 641, 683, 771, 530, 946, 461, 203],         # 6: Trarza
        [1518, 1242, 1031, 843, 767, 641,   0, 908, 1203, 1071, 306, 180, 438],       # 7: Adrar
        [1564, 1289, 1078, 890, 813, 683, 908,   0, 1250, 1120, 1213, 728, 480],        # 8: Dakhlet Nouadhibou
        [1065, 790, 579, 680, 436, 771, 1203, 1250,   0, 910, 1508, 1024, 772],        # 9: Tagant
        [797, 521, 313, 231, 474, 530, 1073, 1120, 910,   0, 1378, 894, 642],      # 10: Guidimakha
        [1823, 1547, 1336, 1148, 1072, 946, 306, 1213, 1508, 1378,   0, 485, 744],    # 11: Tiris Zemmour
        [1339, 1063, 852, 664, 587, 461, 180, 728, 1024, 894, 485,   0, 259],        # 12: Inchiri
        [1087, 811, 600, 412, 336, 203, 438, 480, 772, 642, 744, 259,   0]         # 13: Nouakchott
    ]
    data['num_vehicles'] = 1  # Une seule tournée
    data['depot'] = 0  # Point de départ : Néma (Wilaya 1, index 0 dans Python)
    return data

# Résolution gloutonne (algorithme du plus proche voisin)
def greedy_tsp(distance_matrix):
    """Trouve un itinéraire en utilisant une approche gloutonne."""
    n = len(distance_matrix)
    visited = [False] * n
    route = []
    current_city = 0
    visited[current_city] = True
    route.append(current_city)
    total_distance = 0

    for _ in range(n - 1):
        nearest_city = None
        shortest_distance = float('inf')
        for next_city in range(n):
            if not visited[next_city] and distance_matrix[current_city][next_city] < shortest_distance:
                shortest_distance = distance_matrix[current_city][next_city]
                nearest_city = next_city
        total_distance += shortest_distance
        route.append(nearest_city)
        visited[nearest_city] = True
        current_city = nearest_city

    # Retour à la ville de départ
    total_distance += distance_matrix[current_city][0]
    route.append(0)
    return [r + 1 for r in route], total_distance  # Ajuster l'indexation à partir de 1

# Résolution avec OR-Tools (Approche exacte)
def solve_tsp_with_ortools():
    """Utilise OR-Tools pour résoudre le TSP de manière exacte."""
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Retourne la distance entre deux nœuds."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        distance = data['distance_matrix'][from_node][to_node]
        return distance

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        route = []
        index = routing.Start(0)
        total_distance = 0
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(previous_index))
            total_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        route.append(manager.IndexToNode(index))
        return [r + 1 for r in route], total_distance  # Ajuster pour indexer à partir de 1
    else:
        return None, None

# Comparaison des deux approches
data = create_data_model()
greedy_route, greedy_distance = greedy_tsp(data['distance_matrix'])
exact_route, exact_distance = solve_tsp_with_ortools()

# Résultats
wilayas = [
    "Néma", "Aioun", "Assaba", "Kaedi", "Brakna", "Trarza", "Adrar", "Dakhlet Nouadhibou",
    "Tagant", "Guidimakha", "Tiris Zemmour", "Inchiri", "Nouakchott"
]

print("=== Résultats ===")
print("Approche (gloutonne) :")
greedy_wilayas = [wilayas[i-1] for i in greedy_route]
print(f"Itinéraire : {greedy_wilayas}")
print(f"Distance totale : {greedy_distance} km")
print()
print("Approche exacte (OR-Tools Branch and Bound) :")
exact_wilayas = [wilayas[i-1] for i in exact_route]
print(f"Itinéraire : {exact_wilayas}")
print(f"Distance totale : {exact_distance} km")

=== Résultats ===
Approche (gloutonne) :
Itinéraire : ['Néma', 'Aioun', 'Assaba', 'Brakna', 'Kaedi', 'Guidimakha', 'Trarza', 'Nouakchott', 'Inchiri', 'Adrar', 'Tiris Zemmour', 'Dakhlet Nouadhibou', 'Tagant', 'Néma']
Distance totale : 6231 km

Approche exacte (OR-Tools Branch and Bound) :
Itinéraire : ['Néma', 'Aioun', 'Assaba', 'Tagant', 'Brakna', 'Inchiri', 'Adrar', 'Tiris Zemmour', 'Dakhlet Nouadhibou', 'Nouakchott', 'Trarza', 'Kaedi', 'Guidimakha', 'Néma']
Distance totale : 5800 km
